# Restore a Trained Model
In order to assess your model, you must [pickle its state](train-and-pickle-model.ipnb), so that we can evaluate it using your implementation of `classify()`, which we provide an example using the [linked example model](train-and-pickle-model.ipnb).

Your `classify()` implementation should instantiate your trained model and read the array of matrix glyphs given to it.  You can use the example below to get you started.

We'll start by importing some prerequisites and some code from the course to read test data.  We include a couple sample glyphs from each letter to demonstrate the process.

In [1]:
from six.moves import cPickle as pickle
from scipy import ndimage
import math
import numpy as np
import os
from pathlib import Path
from __future__ import print_function
import tensorflow as tf

In [2]:
%run udacity-tensorflow-support.py

Read images from the samples directory.  Feel free to add your own 28x28 images.

In [3]:
dir_name = 'samples'

glyph_dir = Path.cwd().joinpath(dir_name)
test_folders = [glyph_dir.joinpath(i) for i in glyph_dir.iterdir()]
test_folders.sort()
test_datasets = maybe_pickle(test_folders, 2) # provide only 2 samples for now
test_dataset, test_labels = merge_datasets(test_datasets, 20)
test_dataset, test_labels = randomize(test_dataset, test_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Testing size', test_dataset.shape, test_labels.shape)

C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\A.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\B.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\C.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\D.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\E.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\F.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\G.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\H.pickle already present - Skipping pickling.
C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks\samples\I.pickle already pre

Here is our `classify()` example, with some utility code to help pack the testing data to fit into the pickled tensors.  The labels `test_data:0` and `test_predictor:0` must match those named in the pickled model, [e.g. the example](train-and-pickle-model.ipnb).  Most of the complexity stems from the testing input batch size might not match the training input batch size, so we pad the input tensor.

In [4]:
def pad_data(tensor_shape, dataset=None, labels=None):
    """
    Pad the dataset and labels to fit the pickled input tensor size.
    If tensor_shape is smaller than the input to pad, data are truncated.
    
    Args:
        tensor_shape: 2-element array or tuple of the shape to pad to.
        dataset: the input data to pad, ignored if omitted.
        labels: the labels to pad, ignored if omitted.
    
    Returns:
        The pair of padded data and labels, with the placeholder None if
        the respective input were omitted.
    """
    n = int(tensor_shape[0])
    m = int(dataset.shape[0])
    
    result_data = None
    if dataset is not None:
        result_data = np.ndarray(tensor_shape, dtype=np.float32)
    
    result_labels = None
    if labels is not None:
        result_labels = np.ndarray((n, labels.shape[1]), dtype=np.int32)
        
    for i in range(0, int(math.ceil(n / m))):
        start = i*m
        end = min(n, ((i+1)*m))
        if dataset is not None:
            result_data[start:end, :] = dataset[0:end, :]
        if labels is not None:
            result_labels[start:end, :] = labels[0:end, :]
    return result_data, result_labels


def classify(test_dataset, model_filename='model.meta', checkpoint_path=None):
    """
    A sample classifier to unpickle a TensorFlow model and label the dataset.
    There are magic strings in this function derived from to the model to evaluate.
    Your implementation will likely have different tags that depend upon the model
    implementation.
    
    We pad the input test_dataset to make it at least as large as the model batch,
    and repeat prediction on chunks of the input if it is larger than the model batch.
    
    Args:
        test_dataset: the #observations by 768 feature nd-array glyphs to classify.
        model_filename: optional file name stored by a previous TensorFlow session.
        checkpoint_path: optional path for previous TensorFlow session.
        
    Returns:
        The #observations by #labels nd-array labelings.
    """
    num_classes = 10
    n = int(test_dataset.shape[0])
    result = np.ndarray([n, num_classes], dtype=np.int32)
    
    with tf.Session() as session:
        saver = tf.train.import_meta_graph(model_filename)
        if not checkpoint_path:
            checkpoint_path = Path.cwd()
        saver.restore(session, tf.train.latest_checkpoint(checkpoint_path))
        graph_predict = tf.get_default_graph()
        test_predict = graph_predict.get_tensor_by_name('test_predictor:0') # string from model
        m = int(graph_predict.get_tensor_by_name('test_data:0').shape[0])   # string from model
        
        for i in range(0, int(math.ceil(n / m))):
            start = i*m
            end = min(n, ((i+1)*m))
            test_dataset_pad, _ = pad_data(
                graph_predict.get_tensor_by_name('test_data:0').shape,     # string from model
                dataset=test_dataset[start:end, :])
            result[start:end, :] = test_predict.eval(feed_dict={"test_data:0" : test_dataset_pad})[0:(end-start), :]
    return result[0:test_dataset.shape[0], 0:test_dataset.shape[1]]

And finally, here is how we would use the `classify()` function to evaluate your model.

In [5]:
predict = classify(test_dataset)
print("Test accuracy: %.1f%%" % accuracy(predict, test_labels))

INFO:tensorflow:Restoring parameters from C:\Users\bredi\Documents\Projects\AT-AI-Challenge\notebooks/model
Test accuracy: 50.0%
